## Pre-processing and Visualization

In [7]:
# Import the required libraries.
import math
import random
import numpy as np
import pandas as pd
from string import ascii_lowercase

In [2]:
random.seed(15)
np.random.seed(15)

In [25]:
data = pd.read_csv('../ensemble_data.csv')

In [27]:
letter_map = {letter: int(index) for index, letter in enumerate(ascii_lowercase, start = 1)}

In [28]:
for col in data.columns:
    data[col] = data[col].map(letter_map)

In [29]:
data

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,16,24,19,14,20,16,6,3,14,11,...,19,23,23,16,23,15,16,11,19,21
1,5,24,19,25,20,1,6,3,2,11,...,19,23,23,16,23,15,16,14,14,7
2,5,2,19,23,20,12,6,3,2,14,...,19,23,23,16,23,15,16,14,14,13
3,16,24,25,23,20,16,6,3,14,14,...,19,23,23,16,23,15,16,11,19,21
4,5,24,19,7,6,14,6,23,2,11,...,19,23,23,16,23,15,5,14,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,5,11,19,14,6,14,1,3,2,25,...,19,15,15,16,15,15,16,2,3,12
8120,5,24,19,14,6,14,1,3,2,25,...,19,15,15,16,14,15,16,2,22,12
8121,5,6,19,14,6,14,1,3,2,14,...,19,15,15,16,15,15,16,2,3,12
8122,16,11,25,14,6,25,6,3,14,2,...,11,23,23,16,23,15,5,23,22,12


In [34]:
X = data.loc[:, data.columns != 'type']
Y = data.loc[:, data.columns == 'type']